# Neural Network with JAX

In this lab, we will:
1. Downloads and loads MNIST into NumPy arrays (if it doesn't already
   exist locally).
2. Builds a simple Multi-Layer Perceptron in JAX.
3. Trains the network on MNIST.
4. Evaluates the performance on test data.
5. Provides a custom inference function for your own handwriting
   images.

This lab is based on a
[JAX example](https://github.com/jax-ml/jax/blob/main/examples/mnist_classifier_fromscratch.py).

Please notice that MNIST is the "hello world" for machine learning,
and there are many many examples available online, including some
simplier ones that use libraries:
[JAX with pre-built optimizers](https://github.com/jax-ml/jax/blob/main/examples/mnist_classifier.py),
[FLAX](https://flax.readthedocs.io/en/latest/mnist_tutorial.html),
[pytorch](https://github.com/pytorch/examples/tree/main/mnist), and
[Keras](https://www.tensorflow.org/datasets/keras_example).